In [ ]:
from glob import glob
from pytorchvideo.data.encoded_video import EncodedVideo

In [ ]:
label_paths = glob('./resized/*/*.xml')

In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse(label_paths[0])
root = tree.getroot()

In [ ]:
start_end_frame = list()
for child in root.iter('frame'):
    start_end_frame.append((int(child.find('start').text), int(child.find('end').text)))

In [ ]:
from datetime import datetime

duration = tree.find('./header').findtext('duration')
duration = datetime.strptime(duration, '%H:%M:%S.%f')
duration = duration - datetime(1900, 1, 1)
duration = duration.total_seconds()
total_frame = duration * 30

In [ ]:
start_end_frame.sort(key = lambda x:x[0])
start_end_frame

In [ ]:
total_frame

In [ ]:
def frame_to_sec(frame :int):
    return frame / 30.

In [ ]:
frame_to_sec(start_end_frame[0][0]), frame_to_sec(start_end_frame[0][1])

In [ ]:
def set_image_frame(label_path, event_frames):
    tree = ET.parse(label_path)
    root = tree.getroot()
    
    duration = tree.find('./header').findtext('duration')
    try:
        duration = datetime.strptime(duration, '%H:%M:%S.%f')
        duration = duration - datetime(1900, 1, 1)
        duration = duration.total_seconds()
        total_frame = duration * 30
    except:
        try:
            h, m, s = duration.split(':')
        except:
            m, s = duration.split(':')
            h = 0
        total_frame = float(h) * 3600 + float(m) * 60 + float(s)
    
    normal_frames = list()
    prev_start, prev_end = event_frames[0]
    normal_frames.append((0, prev_start))
    for start, end in event_frames[1:]:
        normal_frames.append((prev_end, start))
    
    start, end = event_frames[-1]
    normal_frames.append((end, int(total_frame)))
    
    final_normal_frames = list()
    for idx, (start, end) in enumerate(normal_frames):
        if end - start >= 30:
            final_normal_frames.append((start, end))
    
    return final_normal_frames

In [ ]:
import os
import copy
from tqdm import tqdm
import cv2
import numpy as np

cut_duration = 3 # sec

fps = 30
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
h, w = EncodedVideo.from_path(label_paths[0].replace('xml', 'mp4')).get_clip(start_sec=frame_to_sec(0), end_sec=frame_to_sec(1))['video'].shape[-2:]
extention = "mp4"

save_path = "./dataset"



for label_path in tqdm(label_paths, total=len(label_paths)):
    if 'swoon' in label_path:
        cls = "swoon"
    else:
        cls = "assult"
    
    tree = ET.parse(label_path)
    root = tree.getroot()
    
    start_end_frames = list()
    for action in root.iter('action'):
        if action.find('actionname').text=='totter':
            continue
        for child in action.iter('frame'):
            start_end_frames.append((int(child.find('start').text), int(child.find('end').text)))
    start_end_frames.sort(key = lambda x:x[0])
    
    prev_start, prev_end = start_end_frames[0]
    new_start, new_end = start_end_frames[0]
    event_frames = list()
    
    flag = False
    for start, end in start_end_frames[1:]:
        if start - prev_end <= 150:
            if not flag:
                flag = True
                new_start = prev_start
            
            new_end = end
        else:
            flag = False
            event_frames.append((new_start-15, new_end+15))
            new_start = start
            new_end = end
        
        prev_start = start
        prev_end = end
    
    event_frames.append((new_start-15, new_end+15))
    
    idx = 0
    for start, end in event_frames:
        if end - start < 30:
            continue
        
        while frame_to_sec(start) + cut_duration < frame_to_sec(end):
            video = EncodedVideo.from_path(label_path.replace('xml', 'mp4')).get_clip(start_sec=frame_to_sec(start), end_sec=frame_to_sec(start)+10)['video']
            print(video.shape)
            video = video.permute(1, 2, 3, 0)
            print(video.shape)
            video = video.numpy().astype(np.uint8)
            
            out = cv2.VideoWriter(f'{save_path}/{cls}/{os.path.basename(label_path)[:-4]}_{cls}_{idx}.{extention}', fourcc, fps, (w, h))
            
            for frame in video:
                out.write(frame)
            out.release()
            
            start += cut_duration * 30
            idx += 1
        
        if end-start >= 30:
            video = EncodedVideo.from_path(label_path.replace('xml', 'mp4')).get_clip(start_sec=frame_to_sec(start), end_sec=frame_to_sec(end))['video']
            video = video.permute(1, 2, 3, 0)
            video = video.numpy().astype(np.uint8)
            out = cv2.VideoWriter(f'{save_path}/{cls}/{os.path.basename(label_path)[:-4]}_{cls}_{idx}.{extention}', fourcc, fps, (w, h))
            for frame in video:
                out.write(frame)

            out.release()
            idx += 1
        
    normal_frames = set_image_frame(label_path, event_frames)
    idx = 0
    for start, end in normal_frames:
        while frame_to_sec(start) + cut_duration < frame_to_sec(end):
            video = EncodedVideo.from_path(label_path.replace('xml', 'mp4')).get_clip(start_sec=frame_to_sec(start), end_sec=frame_to_sec(start)+10)['video']
            video = video.permute(1, 2, 3, 0)
            video = video.numpy().astype(np.uint8)
            out = cv2.VideoWriter(f'{save_path}/normal/{os.path.basename(label_path)[:-4]}_normal_{idx}.{extention}', fourcc, fps, (w, h))
            
            for frame in video:
                out.write(frame)
            out.release()
            
            start += cut_duration * 30
            idx += 1
        
        if end - start >= 30:
            video = EncodedVideo.from_path(label_path.replace('xml', 'mp4')).get_clip(start_sec=frame_to_sec(start), end_sec=frame_to_sec(end))['video']
            video = video.permute(1, 2, 3, 0)
            video = video.numpy().astype(np.uint8)
            out = cv2.VideoWriter(f'{save_path}/normal/{os.path.basename(label_path)[:-4]}_normal_{idx}.{extention}', fourcc, fps, (w, h))
            for frame in video:
                out.write(frame)

            out.release()
            idx += 1